In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import requests
from selenium.webdriver.common.action_chains import ActionChains
products = []
base_url = "https://www.gardena.com"
driver = webdriver.Chrome()
# driver.get("https://www.gardena.com/bg/produkti/polivane/vrazki-za-markuch")





driver.get("https://www.gardena.com/bg/produkti/grija-za-dvora/combisystem")






time.sleep(10)
try:
    # Handling cookie popup if present
    element1 = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
    element1.click()
except:
    pass

try:
    while True:
        # Scroll to the element using JavaScript
        load_more_a = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "a.btn.btn-secondary.icon-end.icon-arrow.more.load-more")))
        driver.execute_script("arguments[0].scrollIntoView(true);", load_more_a)
        driver.execute_script("arguments[0].click();", load_more_a)
        time.sleep(5)  
except:
    pass

try:
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    divs_with_class = soup.find_all('div', class_='col-12 col-md-6 col-lg-3 pb-4')
    print("Product URL:", len(divs_with_class))
    for div in divs_with_class:
        product_link = div.find('a')['href']
        fullproduct_link = urljoin(base_url, product_link)
        products.append(fullproduct_link)
except Exception as e:
    print("Error while getting the product URLs:", e)
finally:
    products = list(set(products))
    driver.quit()

Product URL: 100


In [2]:
print(len(products))


100


In [3]:
desc=[]
title=[]
specs=[]
itemlink=[]
num_lists = 7
img_lists = [[] for _ in range(num_lists)]
driver = webdriver.Chrome()
for link in products:
    driver.get(link)
    time.sleep(15)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    try:
        element1 = driver.find_element(By.XPATH,'//*[@id="onetrust-accept-btn-handler"]')
        element1.click()
    except:
        pass
    
        ############################   GET NAME  #################################
    
    namediv=soup.find('div',class_='col-12 col-lg-6 pt-3 pt-lg-0')
    if namediv:
        name=namediv.find('h1',class_='product-name')
        title.append(name.text)
        print('title done upper',name.text)
    else:
        namediv1=soup.find('div',class_='col-lg-10 col-12 mx-auto px-0')
        if namediv1:
            name=namediv1.find('h2')
            if name:
                title.append(name.text)
                print('title done lower',name.text)
    #             print('---------------',namediv1)
            else:
                print()
                print('------------------------------')
                print(link)
                print('------------link not opened------------------')
                print()
                

                continue
    
##############################  GET DESCRIPTION    ########################################
    try:
        try:
            print('inside try')
            element = driver.find_element(By.CLASS_NAME, 'section-start-content')
            driver.execute_script("arguments[0].scrollIntoView();", element)

            element = driver.find_element(By.CLASS_NAME, 'section-start-content')
            driver.execute_script("arguments[0].scrollIntoView();", element)

            read_more_link = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'read-more_0')))
            time.sleep(3)
            read_more_link.click()            
        except Exception as e:
            print(e)
        time.sleep(5)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        div = soup.find('div', class_='section-start-content pb-lg-8 pb-6 col-12 col-lg-6 ps-lg-0 pe-lg-4 align-self-center')
        if div:
            d = div.get_text(strip=True)
            desc.append(d)
            print('desc done')
        else:
            div = soup.find('div', class_='col-12 col-lg-9 py-4 p-md-10 js-product-desc')
            if div:
                d = div.get_text(strip=True)
                desc.append(d)
                print('desc done in except')
            else:
                print('desv',link)
    except Exception as ex:
        print("An error occurred:",ex)
        


#############################  GET SPECS TABLE   ####################################
#############################  GET SPECS TABLE   ####################################
    try:
        try:
            read_link = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "expand-more-or-less-container"))
            )
            actions = ActionChains(driver)
            actions.move_to_element(read_link).perform()
            driver.execute_script("arguments[0].scrollIntoView(true);", read_link)
            read_link.click()
            print('Specs table clicked')
            time.sleep(5)
        except:
            pass
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')                 
        table = soup.find('div', class_='col-lg-10 col-12 mx-auto')

        if table:
            specs.append(table.text)
        else:
    #             specs.append('N/A')
            pass
        print('specs Done.')
    except:
        pass
    
    
    
    
    response = requests.get(link)

    try:
        src = []
        soup = BeautifulSoup(response.content, 'html.parser')
        splide_track_div = soup.find('div', class_='splide__track')

        if splide_track_div:
            img_elements = splide_track_div.find_all('img', class_='img-fluid')
            print('Images in this product are:', len(img_elements))
            for img in img_elements:
                if img.has_attr('src'):
                    src.append('http:' + img['src'])

        else:
            img_elements = soup.find_all('div', class_='col-12 col-md-3 pb-4 pt-3 d-flex justify-content-center align-items-start')
            print('Images in else are:', len(img_elements))
            for img in img_elements:
                if img.find('img'):
                    img_src = img.find('img').get('src')
                    src.append('http:' + img_src)

        for i in range(min(len(src), num_lists)):
            img_lists[i].append(src[i])

        for i in range(len(src), num_lists):
            img_lists[i].append("")

        print("Images added.")

    except Exception as e:
        print("Error:", e)

    itemlink.append(link)
    print(link)

driver.quit()

title done upper Отделен държач
inside try
Message: element click intercepted: Element is not clickable at point (119, 613)
  (Session info: chrome=123.0.6312.59)
Stacktrace:
	GetHandleVerifier [0x00007FF72BB97072+63090]
	(No symbol) [0x00007FF72BB02CC2]
	(No symbol) [0x00007FF72B99EC65]
	(No symbol) [0x00007FF72B9EBB34]
	(No symbol) [0x00007FF72B9E9954]
	(No symbol) [0x00007FF72B9E7164]
	(No symbol) [0x00007FF72B9E5EF9]
	(No symbol) [0x00007FF72B9DA708]
	(No symbol) [0x00007FF72BA06FDA]
	(No symbol) [0x00007FF72B9DA00A]
	(No symbol) [0x00007FF72BA071F0]
	(No symbol) [0x00007FF72BA23412]
	(No symbol) [0x00007FF72BA06D83]
	(No symbol) [0x00007FF72B9D83A8]
	(No symbol) [0x00007FF72B9D9441]
	GetHandleVerifier [0x00007FF72BF925CD+4238285]
	GetHandleVerifier [0x00007FF72BFCF72D+4488493]
	GetHandleVerifier [0x00007FF72BFC7A0F+4456463]
	GetHandleVerifier [0x00007FF72BC705B6+953270]
	(No symbol) [0x00007FF72BB0E58F]
	(No symbol) [0x00007FF72BB09264]
	(No symbol) [0x00007FF72BB0939B]
	(No symbo

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=123.0.6312.59)
Stacktrace:
	GetHandleVerifier [0x00007FF72BB97072+63090]
	(No symbol) [0x00007FF72BB02CC2]
	(No symbol) [0x00007FF72B99EC65]
	(No symbol) [0x00007FF72B97CA7C]
	(No symbol) [0x00007FF72BA0D687]
	(No symbol) [0x00007FF72BA22AC1]
	(No symbol) [0x00007FF72BA06D83]
	(No symbol) [0x00007FF72B9D83A8]
	(No symbol) [0x00007FF72B9D9441]
	GetHandleVerifier [0x00007FF72BF925CD+4238285]
	GetHandleVerifier [0x00007FF72BFCF72D+4488493]
	GetHandleVerifier [0x00007FF72BFC7A0F+4456463]
	GetHandleVerifier [0x00007FF72BC705B6+953270]
	(No symbol) [0x00007FF72BB0E58F]
	(No symbol) [0x00007FF72BB09264]
	(No symbol) [0x00007FF72BB0939B]
	(No symbol) [0x00007FF72BAF9BD4]
	BaseThreadInitThunk [0x00007FFD1B78257D+29]
	RtlUserThreadStart [0x00007FFD1D1AAA58+40]


In [ ]:
for i in range(len(desc)):
    split_text = desc[i].split('.')
    cleaned_text = '.'.join(split_text[:-1])
    desc[i] = cleaned_text

In [ ]:
print("Length of Description:", len(desc))
print("Length of Title:", len(title))
print("Length of Specifications:", len(specs))
print("Length of products:", len(itemlink))
for i in range(7):
    print(f"Length of Image List {i+1}: {len(img_lists[i])}")

In [ ]:
import pandas as pd
def pad_list(lst, length):
    return lst + [''] * (length - len(lst))
max_length = max(
    len(desc),
    len(title),
    len(specs),
    *[len(img_list) for img_list in img_lists]
)
desc = pad_list(desc, max_length)
title = pad_list(title, max_length)
specs = pad_list(specs, max_length)
img_lists = [pad_list(img_list, max_length) for img_list in img_lists]
data = {
    "Title": title,
    "product_url":itemlink,
    "Description": desc,
    "Specifications": specs,
}
for i in range(7):
    data[f"Image_{i+1}"] = img_lists[i]

# Create DataFrame from the data dictionary
df = pd.DataFrame(data)

# Save DataFrame to a CSV file
# df.to_csv("gardena_CSV.csv", index=False)
df.to_excel("combisystem.xlsx", index=False)

# Print lengths of the lists to verify padding
print("Length of Description:", len(desc))
print("Length of Title:", len(title))
print("Length of Specifications:", len(specs))
for i in range(7):
    print(f"Length of Image List {i+1}: {len(img_lists[i])}")
